# Vector Representations of Words

In [1]:
# -*- coding: utf-8 -*-
    
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
import codecs

In [2]:
f = codecs.open('thai_text','r','utf-8');
words = tf.compat.as_str(f.read()).split()
print('Data size', len(words))

Data size 2459164


In [3]:
print('words = ', words[1:2000]);

words =  ['ออก', 'บล', 'จ.', 'มากสุด', 'เงิน', 'ออก', 'บลจ.', 'มาก', 'สุด', 'ปม', 'ผิด', 'นัดหนี้', 'บีอีพ่นพิษมอนิ่งสตาร์', 'เผย', 'ไตรมาส', 'แรก', 'นี้', 'บลจ.', 'เงิน', 'ไหล', 'ออก', 'มาก', 'สุด', 'หลัง', 'เกิด', 'ภาวะ', 'ผิด', 'นัด', 'ชำระ', 'หนี้', 'ตรา', 'สาร', 'ตั้งแต่', 'ปลาย', 'ปี', 'ก่อน', 'และ', 'นัก', 'ลง', 'ทุน', 'ชะลอ', 'ลง', 'ทุน', 'พบ', 'เงิน', 'ไหล', 'เข้า', 'กอง', 'ทุน', 'รวม', 'สุทธิเพียง', ',', 'ล้าน', 'บาท', 'ลด', 'ลง', 'นายกิตติคุณ', 'ธนรัตนพัฒนกิจ', 'นัก', 'วิเคราะห์', 'กอง', 'ทุน', 'บริษัท', 'มอร์นิ่งสตาร์', 'รีเสิร์ซ', 'ประเทศไทย', 'กล่าว', 'ว่า', 'จาก', 'ปัญหา', 'การ', 'ผิด', 'นัด', 'ชำระ', 'หนี้', 'ของ', 'ตรา', 'สาร', 'หนี้', 'ประเภท', 'หรือ', 'ตรา', 'สาร', 'หนี้', 'ระยะ', 'สั้น', 'ที่', 'เสี่ยง', 'สูง', 'ที่', 'ยังคง', 'มี', 'อยู่', 'ส่ง', 'ผล', 'ต่อ', 'ความ', 'เชื่อมั่น', 'ของ', 'ผู้', 'ลง', 'ทุน', 'จึง', 'เป็น', 'ผล', 'ให้', 'ผู้', 'ลง', 'ทุน', 'ตัดสินใจ', 'ไม่', 'ลง', 'ทุน', 'ต่อ', 'ใน', 'กลุ่ม', 'ดัง', 'กล่าว', 'และ', 'ดู', 'เหมือน', 'ยัง', 'ไม่', 'จบ', 

In [4]:
# Step 2: Build the dictionary and replace rare words with UNK token.
vocabulary_size = 10000

def build_dataset(words, vocabulary_size):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words, vocabulary_size)
del words  # Hint to reduce memory.
print('Most common words (+UNK)', count[:10])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

data_index = 0

Most common words (+UNK) [['UNK', 132064], ('ที่', 67531), ('การ', 56379), ('ใน', 42697), ('และ', 37581), ('มี', 35661), ('ได้', 33724), ('เป็น', 29314), ('ให้', 28738), ('ว่า', 28327)]
Sample data [62, 47, 3000, 1992, 0, 62, 47, 4743, 61, 205] ['เงิน', 'ออก', 'บล', 'จ.', 'UNK', 'เงิน', 'ออก', 'บลจ.', 'มาก', 'สุด']


In [5]:
# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1  # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [skip_window]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  # Backtrack a little bit to avoid skipping words in the end of a batch
  data_index = (data_index + len(data) - span) % len(data)
  return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
  print(batch[i], reverse_dictionary[batch[i]],
        '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

47 ออก -> 62 เงิน
47 ออก -> 3000 บล
3000 บล -> 47 ออก
3000 บล -> 1992 จ.
1992 จ. -> 3000 บล
1992 จ. -> 0 UNK
0 UNK -> 1992 จ.
0 UNK -> 62 เงิน


### Note to Self: Try to run previous command again

In [6]:
# Step 4: Build and train a skip-gram model.

batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():

  # Input data.
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)
    print('embeddings', embeddings);
    print('embed', embed);
    
    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
    print('nce_weights', nce_weights);
    print('nce_biases', nce_biases);

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  loss = tf.reduce_mean(
      tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=train_labels,
                     inputs=embed,
                     num_sampled=num_sampled,
                     num_classes=vocabulary_size))

  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

  # Add variable initializer.
  init = tf.global_variables_initializer()


embeddings <tf.Variable 'Variable:0' shape=(10000, 128) dtype=float32_ref>
embed Tensor("embedding_lookup:0", shape=(128, 128), dtype=float32, device=/device:CPU:0)
nce_weights <tf.Variable 'Variable_1:0' shape=(10000, 128) dtype=float32_ref>
nce_biases <tf.Variable 'Variable_2:0' shape=(10000,) dtype=float32_ref>


In [7]:
# Step 5: Begin training.
num_steps = 30001

with tf.Session(graph=graph) as session:
  # We must initialize all variables before we use them.
  init.run()
  print("Initialized")

  average_loss = 0
  for step in xrange(num_steps):
    batch_inputs, batch_labels = generate_batch(
        batch_size, num_skips, skip_window)
    
#     print('batch_inputs = ', batch_inputs);
#     print('batch_labels = ', batch_labels);
    
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    if step % 2000 == 0:
      if step > 0:
        average_loss /= 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print("Average loss at step ", step, ": ", average_loss)
      average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in xrange(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8  # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k + 1]
        log_str = "Nearest to %s:" % valid_word
        for k in xrange(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log_str = "%s %s," % (log_str, close_word)
        print(log_str)
  final_embeddings = normalized_embeddings.eval()
#   print('sim = ',sim, tf.shape(sim).eval(), tf.rank(sim).eval());
#   print('final_embeddings = ',final_embeddings, tf.shape(final_embeddings).eval(), tf.rank(final_embeddings).eval());

Initialized
Average loss at step  0 :  225.649459839
Nearest to ผ่าน: ด.ช., สงวน, จ่า, สินทรัพย์, การงาน, เมืองเชียงใหม่, พล.อ.อนันตพร, หมายเลขทะเบียน,
Nearest to ๆ: นางแดง, ในเดือนมี.ค., พิธีกร, เป็นอยู่, รีเทนเนอร์, ถา, หาม, หมอวรงค์,
Nearest to มาก: แม่น้ำเมย, ท็อต, พระธาตุศรีสองรัก, ผสมผสาน, พันธมิตร, ภิรมย์ภักดี, ทรัสต์, ดูแลรักษา,
Nearest to เพราะ: จ.กาญจนบุรี, ดาวรุ่ง, ซื้อเล่นรีบาวด์, เบื่อหน่าย, สุขุมวิท, สภาคองเกรส, โรแมนติก, อิ่ม,
Nearest to เมื่อ: ผสาน, จตุจักร, บาห์เรน, อู, เท่าที่, ทัศนคติ, บรรษัท, ตลอดจน,
Nearest to ที่: เฉพาะหน้า, บก.ปอศ., กกล.รส., ประเทศมากขึ้น, ไฟแนนซ์เชียล, องค์การ, สหราชอาณาจักร, ออสเตรเลียน,
Nearest to ทำ: อาจารย์, พระอุโบสถ, แม่ชี, องค์กร, พวกพ้อง, ไฮโล, ทำนอง, จ.สตูล,
Nearest to กล่าว: วิลเลียมส์, ต่อว่า, ยุง, ปลอดทอง, ในหลวง, ผิด, ผัวเมีย, และอีก,
Nearest to เรา: สักขีพยาน, บบ, ฝีมือ, เปื่อย, แทรกซ้อน, เมื่อวันที่, แอสซิสต์, โรงพยาบาลตำรวจ,
Nearest to และ: กรุงปารีส, บริจาค, ตราบ, ประจำปี, ร่ำรวย, จ., ลุ่, เที่ยวแห่งประเทศไทย,
Nearest to แต่: จ.

In [9]:
# Step 6: Visualize the embeddings.

def plot_with_labels(low_dim_embs, labels, filename='tsne_thai.png'):
  assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
  plt.figure(figsize=(18, 18))  # in inches
  for i, label in enumerate(labels):
    x, y = low_dim_embs[i, :]
    plt.scatter(x, y)
    plt.annotate(label,
                 xy=(x, y),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom',
                 fontname='Garuda')

  plt.savefig(filename)

try:
  from sklearn.manifold import TSNE
  import matplotlib
  from matplotlib import rcParams
  matplotlib.rc('font', family='Garuda')
  import matplotlib.pyplot as plt
  rcParams['font.family'] = 'sans-serif'
  rcParams['font.sans-serif'] = ['Garuda']

  tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
  plot_only = 500
  low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
  labels = [reverse_dictionary[i] for i in xrange(plot_only)]
  print(labels);
  print(low_dim_embs);
  plot_with_labels(low_dim_embs, labels)

except ImportError:
  print("Please install sklearn, matplotlib, and scipy to visualize embeddings.")

['UNK', 'ที่', 'การ', 'ใน', 'และ', 'มี', 'ได้', 'เป็น', 'ให้', 'ว่า', 'จะ', 'ของ', 'ไม่', 'มา', 'จาก', 'ความ', 'ไป', 'นี้', 'ผู้', 'กับ', 'ทำ', 'โดย', 'ปี', 'คน', 'วัน', ',', 'ตัว', 'ซึ่ง', 'อยู่', 'ก็', 'ขึ้น', 'ลง', 'ต้อง', 'รับ', 'อย่าง', 'งาน', 'ยัง', 'เข้า', 'เพื่อ', 'แต่', 'กัน', 'ต่อ', 'กล่าว', 'หรือ', 'ทาง', 'ส่วน', 'แล้ว', 'ออก', 'ถึง', 'บาท', 'ด้วย', 'นั้น', 'ใช้', 'ตาม', 'เมื่อ', 'ทั้ง', 'เกิด', 'ผล', 'นำ', 'หลัง', 'รถ', 'มาก', 'เงิน', 'กว่า', 'เรื่อง', 'ๆ', 'ดัง', 'อีก', 'ราย', 'ล้าน', 'ผ่าน', 'ดี', 'นัก', 'เวลา', 'จึง', 'พร้อม', 'รวม', 'ประเทศ', 'ร่วม', 'เพิ่ม', 'ส่ง', 'ขณะ', 'เจ้าหน้าที่', 'ทุน', 'ก่อน', 'พบ', 'ช่วง', 'เพราะ', 'ราคา', 'ครั้ง', 'ถูก', 'สามารถ', 'ไทย', 'เรา', 'เสีย', 'ทุก', 'ด้าน', 'จุด', 'สูง', 'จำนวน', 'คือ', 'เหตุ', 'บ้าน', 'ตลาด', 'เดือน', 'คาด', 'กลุ่ม', 'ขอ', 'ลด', 'น้ำ', 'เห็น', 'ปรับ', 'พื้นที่', 'ตน', 'หา', 'บริษัท', 'ใหม่', 'ค่า', 'ระดับ', 'ฯ', 'กำลัง', 'ประชาชน', 'ข้อ', 'ไว้', 'ขาย', 'ภาย', 'ใหญ่', 'คำ', 'อายุ', 'เมือง', 'ทีม', 'ดำเนิน', 'เปิด', 